In [1]:
import matplotlib.pyplot as plt
import random
import torch
from attack import (
    reconstruct_interactions,
    reconstruct_interactions_with_private_params,
)
from ranker import (
    LinearPDGDRanker,
    Neural1LayerPDGDRanker,
    Neural2LayerPDGDRanker,
    CollaborativeFilteringRecommender,
)
from tqdm import tqdm
from utils import Metrics

In [ ]:
# Simulation

torch.manual_seed(2023)
random.seed(2023)

num_sim_round = 10
num_features = 10
num_data = 1000
lr = 1e-02
max_iters = 1000
num_atk = 10

metrics = Metrics()

models = {
    "linear_pdgd": LinearPDGDRanker(num_features),
    # "neural_1_pdgd": Neural1LayerPDGDRanker(num_features, hidden_size=4),
    # "neural_2_pdgd": Neural2LayerPDGDRanker(
    #     num_features, hidden_size=4, hidden_size2=2
    # ),
}

for _ in tqdm(range(num_sim_round)):
    features = torch.rand(num_data, num_features)
    interactions = torch.randint(0, 2, (num_data,))
    while interactions.sum() == 0:
        interactions = torch.randint(0, 2, (num_data,))
    ranking = list(range(num_data))
    random.shuffle(ranking)
    ranking = torch.LongTensor(ranking)

    for model_name, model in models.items():
        params = model.gen_params()
        log_pos_bias_weight = model.calc_log_pos_bias_weight(
            ranking, model.forward_multiple(params, features), num_data
        )
        
        target = model.grad(
            params,
            features,
            ranking,
            interactions,
            log_pos_bias_weight=log_pos_bias_weight,
        )

        preds_raw = reconstruct_interactions(
            lambda I: model.grad(
                params, features, ranking, I, log_pos_bias_weight=log_pos_bias_weight
            ),
            target,
            num_data,
            lr=lr,
            max_iters=max_iters,
            num_rounds=num_atk,
            return_raw=True,
        )
        preds = preds_raw.sigmoid().round().long()

        metrics.update(model_name, interactions, preds, preds_raw=preds_raw)

In [5]:
print(metrics.get_dataframe().to_string())

             name  accuracy        f1  precision    recall       auc    auc-pr  extra_data
0     linear_pdgd      0.67  0.679612   0.729167  0.636364  0.702222  0.712189         NaN
1   neural_1_pdgd      0.58  0.596154   0.632653  0.563636  0.624242  0.647235         NaN
2   neural_2_pdgd      0.72  0.730769   0.775510  0.690909  0.709495  0.748790         NaN
3     linear_pdgd      0.66  0.638298   0.612245  0.666667  0.717172  0.681038         NaN
4   neural_1_pdgd      0.67  0.666667   0.611111  0.733333  0.695758  0.664537         NaN
5   neural_2_pdgd      0.62  0.586957   0.574468  0.600000  0.651313  0.663541         NaN
6     linear_pdgd      0.61  0.635514   0.641509  0.629630  0.705314  0.711677         NaN
7   neural_1_pdgd      0.60  0.629630   0.629630  0.629630  0.670290  0.711008         NaN
8   neural_2_pdgd      0.58  0.588235   0.625000  0.555556  0.666264  0.702895         NaN
9     linear_pdgd      0.62  0.604167   0.547170  0.674419  0.664627  0.582787         NaN